In [16]:
import os
import pathspec
from pathlib import Path
from datetime import datetime


def build_file_manifest(include_dirs=["/"]):
    project_root = Path().resolve()
    output_dir = project_root / "references"
    output_dir.mkdir(parents=True, exist_ok=True)
    output_filename = output_dir / "file_manifest.txt"

    def load_gitignore_spec(root_dir):
        gitignore_path = root_dir / ".gitignore"
        if gitignore_path.exists():
            return pathspec.PathSpec.from_lines(
                "gitwildmatch", gitignore_path.read_text().splitlines()
            )
        return None

    def traverse_dir(directory, root_dir, spec, include_dirs, indent_level=0):
        lines = []
        indent = "   " * indent_level
        for entry in sorted(os.listdir(directory)):
            if entry in [".git", "__pycache__"]:
                continue
            if (
                indent_level == 1
                and include_dirs != ["/"]
                and entry not in include_dirs
            ):
                continue
            full_path = os.path.join(directory, entry)
            rel_path = os.path.relpath(full_path, root_dir).replace(os.path.sep, "/")
            if spec and spec.match_file(rel_path):
                continue
            lines.append(f"{indent}- {entry}")
            if os.path.isdir(full_path):
                lines.extend(
                    traverse_dir(
                        full_path, root_dir, spec, include_dirs, indent_level + 1
                    )
                )
        return lines

    spec = load_gitignore_spec(project_root)
    tree_lines = [project_root.name]
    tree_lines.extend(
        traverse_dir(project_root, project_root, spec, include_dirs, indent_level=1)
    )

    with output_filename.open("w", encoding="utf-8") as f:
        f.write("\n".join(tree_lines))

    print(
        f"✅ File manifest saved at {datetime.now().isoformat()} to: {output_filename}"
    )


# Run it
build_file_manifest(["/"])  # or use ["app", "experiments"]

✅ File manifest saved at 2025-05-19T00:37:11.274780 to: C:\Repos\codecritic\references\file_manifest.txt


In [17]:
from pathlib import Path


def flatten_code_and_text_files(target_dir: str):
    target_path = Path(target_dir).resolve()
    target_name = target_path.name.lower().replace(" ", "_")
    output_path = Path("references") / f"flattened_{target_name}.txt"
    output_path.parent.mkdir(parents=True, exist_ok=True)

    python_files = []
    text_files = []

    for file in sorted(target_path.rglob("*")):
        if (
            file.is_file()
            and "__pycache__" not in file.parts
            and not file.name.startswith(".")
        ):
            if file.suffix == ".py":
                python_files.append(file)
            elif file.suffix.lower() in [
                ".yaml",
                ".yml",
                ".json",
                ".txt",
                ".md",
                ".ini",
                ".cfg",
            ]:
                text_files.append(file)

    def write_section(header: str, files: list[Path], out):
        out.write(f"\n# === {header.upper()} FILES ===\n\n")
        for f in files:
            out.write(f"# {f.relative_to(target_path)}\n\n")
            try:
                with f.open("r", encoding="utf-8") as source:
                    for i, line in enumerate(source, start=1):
                        sanitized = line.rstrip("\n").replace("#", "# ")
                        out.write(f"# {i} {sanitized}\n")
            except Exception as e:
                out.write(f"# ERROR reading file: {e}\n")
            out.write("\n\n")

    with output_path.open("w", encoding="utf-8") as out:
        write_section("python", python_files, out)
        write_section("non-python", text_files, out)

    print(f"✅ Flattened code and text written to: {output_path}")


dir = ["app", "docs", "experiments", "tests"]
for d in dir:
    # Example usage:
    flatten_code_and_text_files(d)  # Or flatten_code_and_text_files("experiments")

✅ Flattened code and text written to: references\flattened_app.txt
✅ Flattened code and text written to: references\flattened_docs.txt
✅ Flattened code and text written to: references\flattened_experiments.txt
✅ Flattened code and text written to: references\flattened_tests.txt


In [18]:
from pathlib import Path


def ensure_init_files(root_dir: str):
    """
    Recursively ensures all directories under root_dir contain an __init__.py file.

    Args:
        root_dir (str): Path to the directory to scan (e.g., 'agent_utils')
    """
    root_path = Path(root_dir).resolve()
    if not root_path.is_dir():
        print(f"❌ Not a valid directory: {root_path}")
        return

    created = 0
    for folder in root_path.rglob("*"):
        if folder.is_dir():
            init_file = folder / "__init__.py"
            if not init_file.exists():
                init_file.touch()
                created += 1
                print(f"📦 Created: {init_file}")

    print(f"✅ Done. {created} new __init__.py files created under {root_path}.")


# Example usage:
ensure_init_files("app")
# or: ensure_init_files("app")

✅ Done. 0 new __init__.py files created under C:\Repos\codecritic\app.


In [19]:
import sys
from pathlib import Path

from dotenv import load_dotenv

root_env = "env/.env"
load_dotenv(dotenv_path=root_env)

print("✅ Environment loaded from:", root_env)

✅ Environment loaded from: env/.env


In [20]:
import os
import requests

# Get credentials from environment or define manually
SONAR_TOKEN = os.getenv("SONAR_TOKEN")
SONAR_PROJECT_KEY = os.getenv("SONAR_PROJECT_KEY")

# SonarCloud API endpoint
url = "https://sonarcloud.io/api/measures/component"

# Query parameters
params = {
    "component": SONAR_PROJECT_KEY,
    "metricKeys": "complexity,code_smells,comment_lines_density,duplicated_lines_density",
}

# Auth headers
headers = {"Authorization": f"Bearer {SONAR_TOKEN}"}

# Make the request
response = requests.get(url, headers=headers, params=params)

# Raise if error
response.raise_for_status()

# Show parsed JSON
data = response.json()
print(data)

{'component': {'id': 'AZbWimETk2iSI_fLaUTU', 'key': 'benpodraza_codecritic', 'name': 'codecritic', 'qualifier': 'TRK', 'measures': []}}
